In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import urllib
from pyhtml2pdf import converter
import os

baseUrl = 'https://www.cabinetmagazine.org'
baseDirectory = 'D:\CABINET'

indexPage = "index.txt"
magIndex = []

magData = {}

#select range of magazines to pull
for x in range(38,48):
    #print(x)
    
    response = requests.get(baseUrl+ "/issues/" +str(x))
    soup = bs(response.text,'html.parser')
    
    #==================== TITLE
    title = soup.findAll('a',{'class':'link white-only'})
    title = title[0].text.replace("/","") #replace escape characters
    title = title.replace(" ","_") #replace spaces
    title = title.replace("__","_") #replace double underscore
    magIndex.append(title)
    
    folders = []
    
    #=================== ISSUE
    issue = soup.findAll('div',{'class':'issue-date bc-11-1_5'})
    issue = issue[0].text + "\n"
    magIndex.append(issue)  
    
    #=================== GET SECTION HEADINGS
    headings = soup.findAll('h2')
    for heading in headings:
        linkSuffixes = []
        linkTitles = []
        magIndex.append(" - "+heading.text+"\n")        
        folders.append(heading.text)
        item = soup.find('h2', text=heading.text)
        #==================== GET SECTION CONTENT 
        for x in item.next_siblings:
            if x.name == 'h2':
                magIndex.append("--------------------------\n")
                break
            elif hasattr(x, 'text'):
                magIndex.append("-"+x.text)
                #=================== GET THE CONTENT LINKS
                for link in x.find_all('a', href=True):
                    linkSuffixes.append(link['href'])                        
                   
                    string = link.parent.text
                    remove_punctuation_map = dict((ord(char), None) for char in '"”“\',.!@#$%^&*()_+\[\]\{\}\/*?:"<>|')
                    sanitizedTitle = string.translate(remove_punctuation_map )
                    sanitizedTitle = sanitizedTitle.replace("  ","_")
                    sanitizedTitle = sanitizedTitle.replace(" ","_")
                    #print( sanitizedTitle)
                    linkTitles.append(sanitizedTitle)
                #print(linkSuffixes)
                magData[heading.text] = linkSuffixes
                magData[heading.text+"titles"] = linkTitles
          

    indexFilePath = os.path.join(baseDirectory , title ,  indexPage)
    titleFolderPath = os.path.join(baseDirectory , title ) 
    
    if not os.path.exists(titleFolderPath):
        #======== MAKE ISSUE FOLDER
        os.makedirs(titleFolderPath)
        counter = 1
        for folder in folders:
            section = os.path.join(titleFolderPath, str(counter)+"-"+folder)
            section = section.replace(" ","_")
            counter += 1
            if not os.path.exists( section):
                #======== MAKE SECTION FOLDER INSIDE CURRENT ISSUE FOLDER
                os.makedirs(section)
                #======== SAVE PDFS INSIDE EACH SECTION FOLDER
                for i in range(len(magData[folder])):
                    linkSuffix = magData[folder][i]
                    filename = magData[folder + "titles"][i]
                    filename = filename.replace(" ", "_")
                    filepath = os.path.join(baseDirectory, section , filename + ".pdf")
                    filepath = filepath.replace(" ", "_")
                    #======== CREATE AND SAVE PDFS
                    if(linkSuffix.endswith(".php")):
                        if not os.path.isfile(filepath):
                            converter.convert(baseUrl + linkSuffix, filepath)
                            print('created_file : '+ filepath)
                    else:
                            filepath = os.path.join(baseDirectory.strip(), section.strip() , filename.strip() + ".txt")
                            filepath = filepath.replace(" ", "_")
                            index = open(filepath,"w+") 
                            index.write("Non PDF content such as audio / video clips")
                            index.close()
                        
        #======== CREATE AND SAVE INDEX FILE
        if not os.path.isfile(indexFilePath):
            index = open(indexFilePath,"w+", encoding="utf-8") 
            index.write("\n".join(magIndex))
            index.close()
        else:
            print("file exists")
            
    print("done")  